<a href="https://colab.research.google.com/github/solanods/ProjetosFaculdade/blob/main/api_covid_copia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importação das bibliotecas.
import pandas as pd
from pandas.io.sql import to_sql
from sqlalchemy import create_engine, types
import json
import requests
from itertools import chain



# URL para fazer a requisição da API.
url = "https://covid-api.mmediagroup.fr/v1/cases"
# Faz a chamada da API com o método request da bilbioteca requests.
# OBS.: Estamos passando o método GET na chamada da API, isso indica que estamos solicitando somente um retorno de informação.
retorno = requests.request("GET", url)
# Do retorno que foi recebido e jogado em memória dento da variável retorno queremos somente o valor em texto, por isso passamos o parâmetro ".text".
dados = retorno.text
# Carregamos e transformamos o texto para o formato Json com a função loads() da biblioteca json.
dados = json.loads(dados)
# descompactando os valores do dict 'dados.values()' com a função chain do módulo itertools
df = pd.DataFrame(chain.from_iterable(map(lambda sec: sec.values(), dados.values())))
# remover as colunas que não usaremos
df = df.drop(columns=['sq_km_area', 'life_expectancy', 'elevation_in_meters','continent', 'abbreviation', 'iso', 'capital_city'])
#remover as linhas Nan do campo country
df = df.dropna(subset=['country'])
# formatar o campos updated para o YYYY-MM-DD HH:MM:SS
df['updated'] = pd.to_datetime(df['updated'])
# converter coluna "population" to int64 dtype
df = df.astype({"population": int})

# falta fazer!!!
# tratar valores ausentes
# fazer as métricas
# talvez agrupar por location
# após limpos e tratados mandar para o banco

In [ ]:
df

,confirmed,recovered,deaths,country,population,location,lat,long,updated
0,63819,55529,2751,Afghanistan,35530081,Southern and Central Asia,33.93911,67.709953,2021-05-19 16:20:37+00:00
1,132071,126405,2436,Albania,2930187,Southern Europe,41.1533,20.1683,2021-05-19 16:20:37+00:00
2,125693,87609,3388,Algeria,41318142,Northern Africa,28.0339,1.6596,2021-05-19 16:20:37+00:00
3,13569,13234,127,Andorra,76965,Southern Europe,42.5063,1.5218,2021-05-19 16:20:37+00:00
4,31045,26013,685,Angola,29784193,Central Africa,-11.2027,17.8739,2021-05-19 16:20:37+00:00
...,...,...,...,...,...,...,...,...,...
721,4579,2687,37,Vietnam,95540800,Southeast Asia,14.058324,108.277199,2021-05-19 17:20:51+00:00
723,6593,3120,1298,Yemen,28250420,Middle East,15.552727,48.516388,2021-05-19 17:20:51+00:00
724,92630,90919,1263,Zambia,17094130,Eastern Africa,-13.133897,27.849332,2021-05-19 17:20:51+00:00
725,38595,36371,1583,Zimbabwe,16529904,Eastern Africa,-19.015438,29.154857,2021-05-19 17:20:51+00:00


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 178 entries, 0 to 726
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   confirmed   178 non-null    int64              
 1   recovered   178 non-null    int64              
 2   deaths      178 non-null    int64              
 3   country     178 non-null    object             
 4   population  178 non-null    int64              
 5   location    177 non-null    object             
 6   lat         158 non-null    object             
 7   long        158 non-null    object             
 8   updated     157 non-null    datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), int64(4), object(4)
memory usage: 13.9+ KB
